In [1]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    function_tool,
)
load_dotenv()

True

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [2]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)


In [6]:
class GitHubModelProvider(ModelProvider):
    def get_model(self, model_name) -> Model:
        return OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=client)

GITHUB_MODEL_PROVIDER = GitHubModelProvider()

In [7]:
agent = Agent(
    name="Assistant",
    instructions="Answer the user's questions.",

)

User the Runner class to execute the agent 

read the final output for the result of the agent run

In [8]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py
result = await Runner.run(
    agent,
    "Tell me a joke?",
    run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER),
)

print(result.final_output)

Sure! Why did the scarecrow win an award? 

Because he was outstanding in his field!
